In [9]:
from IOHexperimenter import IOH_function, IOH_logger, IOHexperimenter
import numpy as np
import sys
import heapq

In [10]:
budget = 10000

np.random.seed(123)

# hyperparameters
npop = 50 # population size 

In [15]:
def ES_1(problem):
    """
    This function uses:
        - Global Sigma Mutation
        - Intermediate Recombination
        - (mu + lambda) selection
    """
    n = problem.number_of_variables
    
    tao_0 = 1/np.sqrt(n) # learning rate (tao 0)
    lamda = npop * 7 # offspring population size
    
    fopt = -sys.maxsize-1
    
    # Initial Population samples uniformly distributed over the interval (boundaries)
    P = np.random.uniform(low=problem.lowerbound[0],high=problem.upperbound[0], size=(npop, n))
    
    sigma = (problem.upperbound[0]-problem.lowerbound[0])/6 # Global Sigma initialization with feasible range
    
    # Fitness evaluation of the population
    fitness = np.apply_along_axis(problem, 1, P)
    
    if np.max(fitness) >= fopt:
        x_prime = P[np.argmax(fitness)]
        fopt = np.max(fitness)
    
    
    ## !! final_target_hit returns True if the optimum has been found.
    ## !! evaluations returns the number of function evaluations has been done on the problem. 
    while not problem.final_target_hit and problem.evaluations < budget * n: 
        OP = [] # offspring population
        
        ### Intermediate Recombination
        for i in range(lamda):
            parent1 = P[np.random.choice(npop)]
            parent2 = P[np.random.choice(npop)]
            offspring = np.mean((parent1, parent2), axis=0)
            OP.append(offspring)
        
        OP = np.array(OP) # Offspring population
        OP_prime = []
        
        sigma_prime = sigma * np.exp(np.random.normal(0, tao_0))
        
        ### Mutation
        for x_i in OP:
            x_i_prime = x_i + np.random.normal(0, sigma_prime) 
            OP_prime.append(x_i_prime)
            
        OP_prime = np.array(OP_prime)
        
        ### Evaluation of (OP_prime + P)
        total_pop = np.append(P,OP_prime, axis=0)
        fitness = np.apply_along_axis(problem, 1, total_pop)
        if np.max(fitness) >= fopt:
            x_prime = total_pop[np.argmax(fitness)]
            fopt = np.max(fitness)
            
        ### Selection
        top_fits = heapq.nlargest(npop, fitness)
        sorter = np.argsort(fitness)
        indices = sorter[np.searchsorted(fitness, top_fits, sorter=sorter)]
        P = total_pop[indices]

    return x_prime, fopt

In [20]:
def ES_2(problem):
    """
    This function uses:
        - Individual Sigma Mutation
        - Intermediate Recombination
        - (mu + lambda) selection
    """
    n = problem.number_of_variables
    
    tao_prime = 1/np.sqrt(2*n) # global learning rate
    tao_0 = 1/np.sqrt(2*np.sqrt(n)) # local learning rate
    
    lamda = npop * 7 # offspring population size
    
    fopt = -sys.maxsize-1
    
    # Initial Population samples uniformly distributed over the interval (boundaries)
    P = np.random.uniform(low=problem.lowerbound[0],high=problem.upperbound[0], size=(npop, n))
    
    sigmas = [(problem.upperbound[0]-problem.lowerbound[0])/6]*npop # Individual sigma initialization with feasible range
    
    # Fitness evaluation of the population
    fitness = np.apply_along_axis(problem, 1, P)
    
    if np.max(fitness) >= fopt:
        x_prime = P[np.argmax(fitness)]
        fopt = np.max(fitness)
    
    
    ## !! final_target_hit returns True if the optimum has been found.
    ## !! evaluations returns the number of function evaluations has been done on the problem. 
    while not problem.final_target_hit and problem.evaluations < budget * n: 
        OP = [] # offspring population
        
        ### Intermediate Recombination
        for i in range(lamda):
            parent1 = P[np.random.choice(npop)]
            parent2 = P[np.random.choice(npop)]
            offspring = np.mean((parent1, parent2), axis=0)
            OP.append(offspring)
        
        OP = np.array(OP) # Offspring population
        OP_prime = []
        sigmas_prime = []
        
        N_tao_prime = np.random.normal(0, tao_prime)
        
        ### Mutation
        for x_i, sigma_i in zip(OP, sigmas):
            sigma_i_prime = sigma_i * np.exp(N_tao_prime + np.random.normal(0, tao_0))
            sigmas_prime.append(sigma_i_prime)
            
            x_i_prime = x_i + np.random.normal(0, sigma_i_prime) 
            OP_prime.append(x_i_prime)
            
        OP_prime = np.array(OP_prime)
        sigmas = sigmas_prime
        
        ### Evaluation of (OP_prime + P)
        total_pop = np.append(P,OP_prime, axis=0)
        fitness = np.apply_along_axis(problem, 1, total_pop)
        if np.max(fitness) >= fopt:
            x_prime = total_pop[np.argmax(fitness)]
            fopt = np.max(fitness)
            
        ### Selection
        top_fits = heapq.nlargest(npop, fitness)
        sorter = np.argsort(fitness)
        indices = sorter[np.searchsorted(fitness, top_fits, sorter=sorter)]
        P = total_pop[indices]

    return x_prime, fopt

In [21]:
if __name__ == '__main__':

    ## Declarations of Ids, instances, and dimensions that the problems to be tested.
    problem_id = range(1,25)
    instance_id = range(1,26)
    dimension = [2,5,20]

    ## Declariation of IOHprofiler_csv_logger.
    ## 'result' is the name of output folder.
    ## 'studentname1_studentname2' represents algorithm name and algorithm info, which will be caption of the algorithm in IOHanalyzer.
    logger = IOH_logger("./", "result", "ES_2", "ES_2")

    for p_id in problem_id :
        for d in dimension :
            for i_id in instance_id:
                ## Getting the problem with corresponding id,dimension, and instance.
                f = IOH_function(p_id, d, i_id, suite="BBOB")
                f.add_logger(logger)
                xopt, fopt = ES_2(f)
    logger.clear_logger()

KeyboardInterrupt: 